In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [ ]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = False

infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

math_article_seeder = infrastructure_container.math_article_seeder()
math_expression_seeder = infrastructure_container.math_expression_seeder()
math_expression_prediction_seeder = (
    infrastructure_container.math_expression_prediction_seeder()
)
math_article_seeder.seed(reset=RESET)
await math_expression_seeder.seed(reset=RESET)
await math_expression_prediction_seeder.seed(reset=RESET)

math_expression_repository = infrastructure_container.math_expression_repository()
math_expression_prediction_repository = (  # TODO rename
    infrastructure_container.math_expression_prediction_repository()
)
llm = infrastructure_container.llm()